## The LLMs

In [ ]:
from langchain.llms import OpenAI

In [ ]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
llm = OpenAI(model="text-davinci-003", temperature=0.9)

In [ ]:
text = "Suggest a personalized workout routine for someone with thin body & looking to improve cardiovascular endurance and prefers indoor activities."
print(llm(text))

## The Chains

In [2]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="text-davinci-003", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("SUVs"))



MightySUV Motors.


## The Memory

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="text-davinci-003", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)

## Deep Lake Vector Store

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# Before executing the following code, make sure to have your
# Activeloop key saved in the “ACTIVELOOP_TOKEN” environment variable.

# instantiate the LLM and embeddings models
llm = OpenAI(model="text-davinci-003", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
my_activeloop_org_id = "vishnusharma7" 
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Your Deep Lake dataset has been successfully created!


Dataset(path='hub://vishnusharma7/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
   text       text      (2, 1)      str     None   


['b62293a2-2e12-11ee-b60b-847beb0e9772',
 'b62293a3-2e12-11ee-98a7-847beb0e9772']

In [4]:
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever()
)

In [5]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [6]:
response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
 I need to find out when Napoleone was born.
Action: Retrieval QA System
Action Input: When was Napoleone born?
Observation:  Napoleon Bonaparte was born on 15 August 1769.
Thought: I now know the final answer.
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
Napoleon Bonaparte was born on 15 August 1769.


## Agents in LangChain

In [1]:
from langchain.llms import OpenAI

from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper

In [2]:
llm = OpenAI(model="text-davinci-003", temperature=0)

In [3]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

In [4]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

In [5]:
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         max_iterations=6)

In [7]:
response = agent("What's the latest news about the Mars rover?")
print(response['output'])



> Entering new  chain...
 I need to find out the latest news about the Mars rover
Action: google-search
Action Input: "latest news Mars rover"
Observation: The mission has concluded that the solar-powered lander has run out of energy after more than four years on the Red Planet. Dec 15, 2021 ... Mars Sample Return is going to have great stuff to choose from!” Get the Latest JPL News. SUBSCRIBE TO THE NEWSLETTER. Look for the latest news on the Mars Science Laboratory, Curiosity Rover. Curiosity rover finds water-carved 'book' rock on Mars (photo) ... A book-shaped rock on Mars was likely carved by ancient water running through the area, ... Get the latest news releases about the Perseverance rover mission. Oct 19, 2022 ... NASA's Curiosity Mars rover used its Mast Camera, or Mastcam, to capture this panorama of a hill nicknamed ... Get the Latest JPL News. Blog updates written by Mars Perseverance mission team members. ... After our exploration of a boulder field last week, Persevera

## Tools in LangChain

In [8]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

In [9]:
llm = OpenAI(model="text-davinci-003", temperature=0)

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

In [10]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

In [11]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

In [12]:
response = agent("What's the latest news about the Range Rover Sport SV? Then please summarize the results.")
print(response['output'])



> Entering new  chain...
 I need to find the latest news about the Range Rover Sport SV and then summarize it.
Action: Search
Action Input: "Range Rover Sport SV news"
Observation: RANGE ROVER SV CARMEL EDITION MAKES ITS DEBUT AT RANGE ROVER HOUSE. (MAHWAH, N.J.) – August 19, 2022 – Jaguar Land Rover North America revealed the Ultra- ... May 12, 2023 ... Land Rover redesigned the Range Rover Sport for 2023, and soon the SUV's high-performance variant developed by the SVO special projects ... May 31, 2023 ... The impressive figure from the 4.4-liter V-8 makes this the most powerful Range Rover Sport ever, with an estimated sprint to 60 mph in 3.6 ... Get the latest on new Land Rover Range Rover Sport models including first drives, in-depth reviews, concept car photos, new features, recalls and more. May 30, 2023 ... The Range Rover Sport SV replaces the old SVR. You remember that, of course: often painted bright blue, made a noise like Hercules gargling ... May 31, 2023 ... Performanc

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UI2XgHYJMlcnd4QY1Nlbp1s6 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-UI2XgHYJMlcnd4QY1Nlbp1s6 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/acco

 I now know the final answer.
Final Answer: Jaguar Land Rover North America has revealed the Range Rover Sport SV Carmel Edition, the high-performance variant of the Range Rover Sport SUV. Developed by the SVO special projects team, the Range Rover Sport SV is the most powerful Range Rover Sport ever, with a 4.4-liter V-8 engine that can reach 60 mph in 3.6 seconds. It features a unique exterior design, 23-inch carbon fibre wheels, and a noise like Hercules gargling. It is the performance flagship of the Range Rover Sport, combining sporting luxury with innovative performance-enhancing technologies.

> Finished chain.
Jaguar Land Rover North America has revealed the Range Rover Sport SV Carmel Edition, the high-performance variant of the Range Rover Sport SUV. Developed by the SVO special projects team, the Range Rover Sport SV is the most powerful Range Rover Sport ever, with a 4.4-liter V-8 engine that can reach 60 mph in 3.6 seconds. It features a unique exterior design, 23-inch car